In [5]:
from Tiny-ImageNet-10000 import get_data

train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

SyntaxError: invalid syntax (<ipython-input-5-b110a60a97ca>, line 1)